In [8]:
# Step 1: Install required packages
# Using langchain-huggingface and langchain-chroma for up-to-date compatibility
print("Installing packages...")
!pip install -qU langchain langchain-community langchain-huggingface langchain-chroma sentence-transformers chromadb transformers torch accelerate bitsandbytes
# Reinstalling torchvision to resolve potential conflicts or corruption
!pip install --upgrade --force-reinstall torchvision


Installing packages...
  Using cached torch-2.10.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (31 kB)
  Using cached cuda_bindings-12.9.4-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (2.6 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cach

In [9]:
!pip install -qU langchain-classic

In [4]:
# Step 2: Import libraries
import torch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.schema import Document


print("Starting Complete RAG Pipeline Demo")
print("="*60)

Starting Complete RAG Pipeline Demo


In [5]:
from transformers import AutoTokenizer

In [6]:
from transformers import AutoModelForSeq2SeqLM

In [17]:
!pip install -q huggingface-hub==0.36.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 0.36.2
    Uninstalling huggingface_hub-0.36.2:
      Successfully uninstalled huggingface_hub-0.36.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.2.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.0 which is incompatible.


In [2]:
from transformers.pipelines import pipeline

In [7]:
# Step 3: Create sample documents
print("\nStep 1: Creating knowledge base...")
documents = [
    Document(page_content="LangChain is a framework for developing applications powered by language models. It provides tools for document loading, splitting, embeddings, and chains.", metadata={"source": "doc1"}),
    Document(page_content="RAG stands for Retrieval-Augmented Generation. It combines retrieval of relevant documents with text generation. First, relevant documents are retrieved based on similarity, then an LLM generates an answer using those documents as context.", metadata={"source": "doc2"}),
    Document(page_content="Vector databases store embeddings and allow for semantic similarity search. They convert text into numerical vectors that capture meaning.", metadata={"source": "doc3"}),
    Document(page_content="ChromaDB is an open-source embedding database that works well with LangChain. It's lightweight and perfect for development.", metadata={"source": "doc4"}),
    Document(page_content="HuggingFace provides thousands of pre-trained models that can run locally or in the cloud. Popular models include FLAN-T5, GPT-2, and Llama.", metadata={"source": "doc5"}),
    Document(page_content="Embeddings are numerical representations of text that capture semantic meaning. Similar texts have similar embeddings.", metadata={"source": "doc6"}),
]


Step 1: Creating knowledge base...


In [8]:
# Step 4: Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(documents)
print(f"Created {len(splits)} document chunks")

Created 6 document chunks


In [9]:
# Step 5: Create embeddings
print("\nStep 2: Loading embedding model...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



Step 2: Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Step 6: Create vector store
print("\nStep 3: Creating vector database...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="rag_demo"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})



Step 3: Creating vector database...


In [11]:
# Step 7: Load LLM (FLAN-T5 Large)
print("\nStep 4: Loading Language Model (FLAN-T5 Large)...")
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device=0 if torch.cuda.is_available() else -1,
    model_kwargs={"temperature": 0.1, "do_sample": False} # Lower temp for more factual RAG
)
llm = HuggingFacePipeline(pipeline=pipe)



Step 4: Loading Language Model (FLAN-T5 Large)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [13]:
# Step 8: Create RAG Chain (Modern LCEL approach)
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Keep the answer concise.\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [14]:
# Step 9: Test Function
def ask_question(query):
    print(f"\nQuestion: {query}")
    print("Step 1: Retrieving & Generating...")

    # Modern LangChain uses 'input', returns 'answer' and 'context'
    response = rag_chain.invoke({"input": query})

    print(f"Retrieved {len(response['context'])} documents.")
    print(f"\nAnswer: {response['answer']}")
    print("-" * 60)


In [15]:
# Run demo queries
print("\nRunning Demo Queries...\n")
ask_question("What is RAG?")


Running Demo Queries...


Question: What is RAG?
Step 1: Retrieving & Generating...
Retrieved 2 documents.

Answer: Retrieval-Augmented Generation
------------------------------------------------------------


In [16]:
ask_question("What are embeddings and why are they useful?")


Question: What are embeddings and why are they useful?
Step 1: Retrieving & Generating...
Retrieved 2 documents.

Answer: numerical representations of text that capture semantic meaning
------------------------------------------------------------


In [17]:
ask_question("Which database should I use for vector storage?")


Question: Which database should I use for vector storage?
Step 1: Retrieving & Generating...
Retrieved 2 documents.

Answer: ChromaDB
------------------------------------------------------------
